# 第九章 符合Python风格的对象

## 9.1 对象表示形式

* repr()    以便开发者理解的方式返回对象的字符串表示形式
* str()     以便用户理解的方式返回对象的字符串表示形式

## 9.3 备选构造方法

In [4]:
from array import array
import math


class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)  # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


## 9.4 classmethod 和 staticmethed 

示例 9-4 比较classmethod和staticmethod的行为

In [5]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args
Demo.klassmeth(), Demo.klassmeth("spam"), Demo.statmeth(), Demo.statmeth("spam")

((__main__.Demo,), (__main__.Demo, 'spam'), (), ('spam',))

## 9.5 格式化显示

In [8]:
brl = 1/2.43
brl, format(brl, "0.4f"), '1BRL = {rate:0.2f} USD'.format(rate=brl)

(0.4115226337448559, '0.4115', '1BRL = 0.41 USD')

## 9.6 可散列的Vector2d

In [9]:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [10]:
set([v1])

TypeError: unhashable type: 'Vector2d'

In [11]:
v1.x, v1.y

(3.0, 4.0)

In [13]:
v1.x = 7
v1.x, v1.y

(7, 4.0)

示例 9-7 让Vector2d 不可变

In [15]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


## 9.7 Python 的 私有属性和受保护的属性

In [16]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [17]:
v1._Vector2d__x

3.0

## 9.8 使用 `__slots__`类属性节省空间

## 9.10 本章小结

<img src="./images/第九章总结1.jpg" width="70%">
<img src="./images/第九章总结2.jpg" width="70%">